In [3]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


You should consider upgrading via the 'c:\users\bouch\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [14]:
!pip install tweepy==3.10.0

You should consider upgrading via the 'c:\users\bouch\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.5.0
    Uninstalling tweepy-4.5.0:
      Successfully uninstalled tweepy-4.5.0


In [1]:
import tweepy
from tweepy import StreamListener
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from tweepy import OAuthHandler
from kafka import KafkaProducer
import json

In [2]:
producer = KafkaProducer(bootstrap_servers='localhost:29092')

In [3]:
topic_name = "tweet-data"

In [4]:
access_token = "4518193835-XGOBCZibmnUdgkPZM3CGLL6JFKqoAktx1VimWYD"
access_secret = "GQrGPwhJehHwhQiw7pTC4CzXMioRLBFNWSpiwqIzkZAjA"
consumer_key = "XRuZQD2Sq8Ojtvo3dbe1Z2U0M"
consumer_secret = "FRjWoFHTSxWKIYfRZCsifD6jRJRjTJBunet8JUj4pJOiZp2y4x"

In [5]:
class twitterAuth():

    def authenticateTwitterApp(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        
        # auth = tweepy.OAuth2BearerHandler("xxxx")
        # auth = tweepy.API(auth)

        return auth


In [6]:
class TwitterStreamer():

    def __init__(self):
        self.twitterAuth = twitterAuth()

    def stream_tweets(self):
        while True:
            try:
                listener = TweetsListener() 
                auth = self.twitterAuth.authenticateTwitterApp()
                stream = Stream(auth, listener)
                stream.filter(track=["project_test"], stall_warnings=True, languages= ["en"])
            except Exception as e:
                print(e)

In [7]:
class TweetsListener(StreamListener):
    
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      
      if "extended_tweet" in msg:
        out_data = '{ "tweet":"'+str(msg['extended_tweet']['full_text']).replace("\n","")+'","user":"'+str(msg['user']['screen_name'])+'", "tweet_id":"'+str(msg['id_str'])+'" }'
        print(out_data)
        producer.send(topic_name, str.encode(out_data))
      else:
        out_data = '{ "tweet":"'+str(msg['text']).replace("\n","")+'","user":"'+str(msg['user']['screen_name'])+'", "tweet_id":"'+str(msg['id_str'])+'" }'
        print(out_data)
        producer.send(topic_name, str.encode(out_data))
      return True

    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True


  def on_error(self, status):
    print(status)
    return True

  def on_exception(self, exception):
        print(exception)
        return

In [ ]:
if __name__ == "__main__":
    TS = TwitterStreamer() 
    TS.stream_tweets()

new message
{ "tweet":"@project_test i have asked for a larger leg space seats but got the other one and now stuck for three hourse from casa to tangier.","user":"parz1_", "tweet_id":"1494253581914058755" }
('Connection broken: IncompleteRead(128 bytes read)', IncompleteRead(128 bytes read))
('Connection broken: IncompleteRead(128 bytes read)', IncompleteRead(128 bytes read))
new message
{ "tweet":"@project_test i have asked for larger leg space seats but got the other one and now stuck for 3 hourse from tangier to casa","user":"parz1_", "tweet_id":"1494260593133555715" }
new message
{ "tweet":"@Project_test i have asked for larger leg space seats but got the other one and now stuck for 3 hourse from Tangier to Casablanca","user":"parz1_", "tweet_id":"1494272450997194757" }
new message
{ "tweet":"@Project_test i have asked for larger leg space seats but got the other one and now stuck for 3 hours from Tangier to Casablanca","user":"parz1_", "tweet_id":"1494275050924654593" }
new messag